b简答题：
1. 如果你的训练集具有数百万个特征，那么可以使用哪种线性回归训练算法？

2. 如果你的训练集里特征的数值大小迥异，那么哪些算法可能会受到影响？受影响程度如何？你应该怎么做？

3. 训练逻辑回归模型时，梯度下降可能会卡在局部最小值中吗？

4. 如果你让它们运行足够长的时间，是否所有的梯度下降算法都能得出相同的模型？

5. 假设你使用批量梯度下降，并在每个轮次绘制验证误差。如果你发现验证错误持续上升，那么可能是什么情况？你该如何解决？

6. 当验证误差上升时立即停止小批量梯度下降是个好主意吗？

7. 哪种梯度下降算法（在我们讨论过的算法中）将最快地到达最佳解附近？哪个实际上会收敛？如何使其他的也收敛

8. 假设你正在使用多项式回归。绘制学习曲线后，你会发现训练误差和验证误差之间存在很大的差距。发生了什么？解决此问题的三种方法是什么？

9. 假设你正在使用岭回归，并且你注意到训练误差和验证误差几乎相等且相当高。你是否会说模型存在高偏差或高方差？你应该增加正则化超参数α还是减小它呢？

10. 为什么要使用：a.岭回归而不是简单的线性回归（即没有任何正则化）？b.Lasso而不是岭回归？c.弹性网络而不是Lasso回归？

11. 假设你要将图片分类为室外/室内和白天/夜间。你应该实现两个逻辑回归分类器还是一个softmax回归分类器？

In [ ]:
# 1:随机梯度下降
# 2:梯度下降、SVM:导致模型收敛缓慢:对特征进行标准化
# 3:不会
# 4:不一定
# 5:过拟合:引入正则化，减少特征数量，采用早停机制，增加训练数据
# 6:不是
# 7:随机梯度下降:批量梯度下降：小批量梯度下降
# 8：过拟合：简化模型，增加正则化，增加训练数据量
# 9：模型过拟合：减小a
# 10:a. ：岭回归通过 L2 正则化限制权重大小，解决简单线性回归的过拟合问题，提高模型泛化能力
# b. Lasso 使用 L1 正则化，可将不重要特征的权重压缩至 0，实现自动特征选择；而岭回归仅缩小权重，不剔除特征。
# c.：弹性网络结合 L1 和 L2 正则化，解决 Lasso 在特征高度相关时选择不稳定的问题，同时保留特征选择能力
# 11:两个逻辑回归

编程题：

In [183]:
# todo 编程题: 在不使用sklearn的情况下，仅使用Numpy，为softmax回归实现带早停的批量梯度下降，将它用于分类任务，例如鸢尾花数据集

#  注意：
#  1. 要实现l2正则化
#  2. 除了数据读取，其他仅使用numpy，包括训练集+验证集分离，以及softmax预测 和 损失计算

In [2]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, dual=True, random_state=42))])

In [6]:
import numpy as np  # 导入NumPy库，用于数值计算
from sklearn.datasets import load_iris  # 仅导入数据集加载函数，其他功能不用

# 加载鸢尾花数据集
iris = load_iris()  # 加载完整的鸢尾花数据集
X = iris.data[:, [2, 3]]  # 提取特征矩阵，只保留第2和3列（花瓣长度和宽度）
y = (iris.target == 2)  # 将标签转为二元分类问题：是否为第2类（1表示是，0表示否）

# 划分训练集和验证集（8:2比例）
np.random.seed(42)  # 设置随机种子，保证结果可重现
indices = np.random.permutation(len(X))  # 生成打乱的索引，用于随机划分数据
split_idx = int(0.8 * len(X))  # 计算划分点，80%数据作为训练集
# 按照索引划分训练集和验证集的特征
X_train, X_val = X[indices[:split_idx]], X[indices[split_idx:]]
# 按照索引划分训练集和验证集的标签
y_train, y_val = y[indices[:split_idx]], y[indices[split_idx:]]

# 特征标准化（使特征均值为0，标准差为1）
mean = np.mean(X_train, axis=0)  # 计算训练集每个特征的均值
std = np.std(X_train, axis=0)  # 计算训练集每个特征的标准差
std[std == 0] = 1  # 处理标准差为0的情况，避免除以0错误
X_train_scaled = (X_train - mean) / std  # 标准化训练集特征
X_val_scaled = (X_val - mean) / std  # 使用训练集的均值和标准差标准化验证集（重要！）

# 初始化模型参数
n_features = X_train_scaled.shape[1]  # 获取特征数量（这里是2）
# 随机初始化权重，乘以0.01是为了让初始值较小
weights = np.random.randn(n_features) * 0.01
bias = 0.0  # 初始化偏置为0

# 设置超参数
learning_rate = 0.1  # 学习率，控制参数更新幅度
lambda_reg = 0.01  # 正则化系数，控制正则化强度
batch_size = 16  # 批量大小，每次迭代使用的样本数
epochs = 1000  # 最大训练轮数
patience = 20  # 早停耐心值，验证损失不改善的最大轮数

# 初始化训练过程中需要记录的变量
train_loss_history = []  # 存储每轮训练的训练集损失
val_loss_history = []  # 存储每轮训练的验证集损失
best_val_loss = float('inf')  # 记录最佳验证损失，初始化为无穷大
best_weights = weights.copy()  # 存储最佳权重参数
best_bias = bias  # 存储最佳偏置参数
early_stop_counter = 0  # 早停计数器，记录验证损失不改善的轮数

# 批量梯度下降训练循环
for epoch in range(epochs):  # 遍历每个训练轮次
    # 打乱训练数据顺序（每个epoch都打乱，增加随机性）
    shuffled_indices = np.random.permutation(len(X_train_scaled))
    X_shuffled = X_train_scaled[shuffled_indices]  # 打乱后的训练特征
    y_shuffled = y_train[shuffled_indices]  # 打乱后的训练标签

    # 按批量更新参数
    for i in range(0, len(X_shuffled), batch_size):  # 遍历每个批次
        X_batch = X_shuffled[i:i+batch_size]  # 当前批次的特征
        y_batch = y_shuffled[i:i+batch_size]  # 当前批次的标签

        # 前向传播计算预测值
        z = np.dot(X_batch, weights) + bias  # 计算线性输出 z = X·w + b
        # 计算sigmoid激活值（二元分类的概率），np.clip防止指数溢出
        y_pred = 1 / (1 + np.exp(-np.clip(z, -250, 250)))

        # 计算梯度
        batch_size_current = X_batch.shape[0]  # 当前批次的实际样本数（最后一批可能不足batch_size）
        # 权重梯度：包含数据梯度和L2正则化梯度
        dw = (1 / batch_size_current) * np.dot(X_batch.T, (y_pred - y_batch)) + lambda_reg * weights
        # 偏置梯度：只包含数据梯度（偏置不参与正则化）
        db = (1 / batch_size_current) * np.sum(y_pred - y_batch)

        # 更新参数（梯度下降）
        weights -= learning_rate * dw  # 权重更新
        bias -= learning_rate * db  # 偏置更新

    # 计算本轮训练后的训练集损失
    z_train = np.dot(X_train_scaled, weights) + bias  # 训练集线性输出
    # 训练集预测概率
    y_pred_train = 1 / (1 + np.exp(-np.clip(z_train, -250, 250)))
    # 计算二元交叉熵损失（添加1e-10防止log(0)错误）
    cross_entropy_train = -np.mean(y_train * np.log(y_pred_train + 1e-10) +
                                  (1 - y_train) * np.log(1 - y_pred_train + 1e-10))
    l2_loss_train = 0.5 * lambda_reg * np.sum(weights **2)  # L2正则化损失
    train_loss = cross_entropy_train + l2_loss_train  # 总训练损失
    train_loss_history.append(train_loss)  # 记录训练损失

    # 计算本轮训练后的验证集损失
    z_val = np.dot(X_val_scaled, weights) + bias  # 验证集线性输出
    # 验证集预测概率
    y_pred_val = 1 / (1 + np.exp(-np.clip(z_val, -250, 250)))
    # 计算验证集交叉熵损失
    cross_entropy_val = -np.mean(y_val * np.log(y_pred_val + 1e-10) +
                                (1 - y_val) * np.log(1 - y_pred_val + 1e-10))
    l2_loss_val = 0.5 * lambda_reg * np.sum(weights** 2)  # 验证集L2正则化损失
    val_loss = cross_entropy_val + l2_loss_val  # 总验证损失
    val_loss_history.append(val_loss)  # 记录验证损失


# 评估最佳模型在训练集上的表现
z_train_best = np.dot(X_train_scaled, best_weights) + best_bias  # 使用最佳参数计算训练集输出
# 转换为类别预测（概率>=0.5为1，否则为0）
y_pred_train_best = (1 / (1 + np.exp(-np.clip(z_train_best, -250, 250))) >= 0.5).astype(int)
train_acc = np.mean(y_pred_train_best == y_train)  # 计算训练集准确率

# 评估最佳模型在验证集上的表现
z_val_best = np.dot(X_val_scaled, best_weights) + best_bias  # 使用最佳参数计算验证集输出
# 转换为类别预测
y_pred_val_best = (1 / (1 + np.exp(-np.clip(z_val_best, -250, 250))) >= 0.5).astype(int)
val_acc = np.mean(y_pred_val_best == y_val)  # 计算验证集准确率

# 输出最终结果
print(f"\n训练集准确率: {train_acc:.4f}")  # 打印训练集准确率
print(f"验证集准确率: {val_acc:.4f}")  # 打印验证集准确率
print(f"最优权重: {best_weights}")  # 打印最佳权重参数
print(f"最优偏置: {best_bias}")  # 打印最佳偏置参数



训练集准确率: 0.2500
验证集准确率: 0.3333
最优权重: [-0.00534416 -0.01495054]
最优偏置: 0.0
